In [6]:
import numpy as np
import pandas as pd
from fancyimpute import IterativeImputer
from sklearn.preprocessing import StandardScaler


In [7]:
'''
pre-processes melanoma data

'''

'\npre-processes melanoma data\n\n'

In [8]:
# read in data

raw_data = pd.read_csv('../Melanoma/total_PD_1.csv')
raw_data.head()

/Users/sophiesun/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Unnamed: 0,GSM2445716,GSM2445717,GSM2445718,GSM2445719,GSM2445720,GSM2445721,GSM2445722,GSM2445723,GSM2445724,...,SRR5088909,SRR5088911,SRR5088913,SRR5088914,SRR5088916,SRR5088920,SRR5088922,SRR5088924,SRR5088926,SRR5088929
0,5S_rRNA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.039753,0.1036,0,0,0.0428182,0.138019,0.0139869,0.0539995,0.0914821,0.0727011
1,5_8S_rRNA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
2,7SK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
3,A1BG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0298032,0.244701,0.172904,0.10584,0.188846,0.102993,0.0440417,0.151334,0.169748,0.0304227
4,A1BG-AS1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.410368,1.10834,1.55227,1.59396,1.20657,0.951995,0.117314,0.550993,1.18046,0.645802


In [9]:
# some re-formatting

# rename column

raw_data = raw_data.rename(columns={raw_data.columns[0]: 'patient'})

# transpose so columns are genes and rows are patients

t_data = raw_data.set_index('patient').T
print(t_data.head())

# split data

labels = t_data['Response']
clean_data = t_data.drop(columns = ['Response'])


patient    5S_rRNA 5_8S_rRNA  7SK A1BG A1BG-AS1 A1CF      A2M A2M-AS1 A2ML1  \
GSM2445716     NaN       NaN  NaN  NaN      NaN  NaN  12.4884     NaN   NaN   
GSM2445717     NaN       NaN  NaN  NaN      NaN  NaN  11.4637     NaN   NaN   
GSM2445718     NaN       NaN  NaN  NaN      NaN  NaN  12.1104     NaN   NaN   
GSM2445719     NaN       NaN  NaN  NaN      NaN  NaN  13.3247     NaN   NaN   
GSM2445720     NaN       NaN  NaN  NaN      NaN  NaN  14.4241     NaN   NaN   

patient    A2ML1-AS1  ... snoU2_19 snoU83B snoZ196 snoZ278 snoZ40 snoZ6  \
GSM2445716       NaN  ...      NaN     NaN     NaN     NaN    NaN   NaN   
GSM2445717       NaN  ...      NaN     NaN     NaN     NaN    NaN   NaN   
GSM2445718       NaN  ...      NaN     NaN     NaN     NaN    NaN   NaN   
GSM2445719       NaN  ...      NaN     NaN     NaN     NaN    NaN   NaN   
GSM2445720       NaN  ...      NaN     NaN     NaN     NaN    NaN   NaN   

patient    snosnR66 uc_338 yR211F11.2 Response  
GSM2445716      NaN    Na

In [10]:
# drop any genes with > 90% missing data

clean_data = clean_data.dropna(thresh = clean_data.shape[0]*0.9, how='all', axis=1)
clean_data.head()

patient,A2M,ABCB1,ABL1,ADA,ADORA2A,AICDA,AIRE,AKT3,ALCAM,AMBP,...,ULBP2,USP9Y,VCAM1,VEGFA,VEGFC,XCL2,XCR1,YTHDF2,ZAP70,ZNF205
GSM2445716,12.4884,7.016,9.38074,6.82087,4.85041,2.64096,3.3414,11.309,7.60096,2.96289,...,6.067222319,1.96288566,6.95384052,9.189779473,6.800828901,3.100389183,3.96288566,10.79133953,5.49893856,7.033274987
GSM2445717,11.4637,5.43465,8.65705,6.30912,4.61523,3.11273,4.2282,12.6538,9.57216,2.11273,...,4.228202656,6.794549479,6.794549479,7.434653534,6.228202656,4.434653534,3.69768794,10.21251156,7.636287395,6.142472782
GSM2445718,12.1104,5.66996,9.85013,8.10566,6.16592,2.96952,3.55448,10.5177,5.84399,2.70649,...,5.291449305,0.384558709,6.999268553,7.724408712,6.191913631,2.384558709,2.706486804,10.0284149,6.554483711,6.554483711
GSM2445719,13.3247,6.47553,8.68792,8.26729,5.91365,3.52134,3.52134,11.2229,7.77558,2.74373,...,3.521337443,2.936374942,8.332979723,8.184302455,6.379318438,6.276224945,4.258303037,10.49575203,8.498617366,5.913654865
GSM2445720,14.4241,7.22529,9.96367,8.12778,7.74927,4.21583,4.55287,11.3331,8.26711,3.77526,...,5.552869187,3.427338305,9.404618228,9.841350799,7.001329688,6.410850182,5.360224109,9.682152204,8.987497414,6.253308905


In [62]:
# log2 transformation

# note that samples from the Melanoma dataset GSE931 are already log2 transformed

# add 1 to all values for log2 transformation

clean_data = clean_data.applymap(lambda x: x+1 if isinstance(x, (int, float)) else x)

# subset GSE931 samples from rest

gse931 = clean_data[clean_data.index.str.contains('GSM24457')]
to_transform = clean_data[~clean_data.index.str.contains("GSM24457")]

# log2 transform 

log2_data = to_transform.applymap(lambda x: np.log2(x) if isinstance(x, (int, float)) else x)

# re-join data

log2_data = pd.concat([gse931, log2_data], axis = 0)
log2_data.head()


patient,A2M,ABCB1,ABL1,ADA,ADORA2A,AICDA,AIRE,AKT3,ALCAM,AMBP,...,ULBP2,USP9Y,VCAM1,VEGFA,VEGFC,XCL2,XCR1,YTHDF2,ZAP70,ZNF205
GSM2445716,15.488406,10.015997,12.380738,9.820867,7.850411,5.640958,6.341397,14.308956,10.600959,5.962886,...,6.067222319,1.96288566,6.95384052,9.189779473,6.800828901,3.100389183,3.96288566,10.79133953,5.49893856,7.033274987
GSM2445717,14.463665,8.434654,11.657046,9.309123,7.615226,6.112725,7.228203,15.653822,12.572157,5.112725,...,4.228202656,6.794549479,6.794549479,7.434653534,6.228202656,4.434653534,3.69768794,10.21251156,7.636287395,6.142472782
GSM2445718,15.110351,8.669961,12.850125,11.105658,9.165918,5.969521,6.554484,13.517701,8.843990,5.706487,...,5.291449305,0.384558709,6.999268553,7.724408712,6.191913631,2.384558709,2.706486804,10.0284149,6.554483711,6.554483711
GSM2445719,16.324661,9.475534,11.687919,11.267292,8.913655,6.521337,6.521337,14.222933,10.775579,5.743730,...,3.521337443,2.936374942,8.332979723,8.184302455,6.379318438,6.276224945,4.258303037,10.49575203,8.498617366,5.913654865
GSM2445720,17.424133,10.225295,12.963674,11.127778,10.749266,7.215834,7.552869,14.333136,11.267115,6.775262,...,5.552869187,3.427338305,9.404618228,9.841350799,7.001329688,6.410850182,5.360224109,9.682152204,8.987497414,6.253308905


In [63]:
# missing data imputation

imputer = IterativeImputer()
imputed_data = imputer.fit_transform(log2_data)

imputed_data = pd.DataFrame(imputed_data, columns = log2_data.columns, index = log2_data.index)
imputed_data.head()

patient,A2M,ABCB1,ABL1,ADA,ADORA2A,AICDA,AIRE,AKT3,ALCAM,AMBP,...,ULBP2,USP9Y,VCAM1,VEGFA,VEGFC,XCL2,XCR1,YTHDF2,ZAP70,ZNF205
GSM2445716,15.488406,10.015997,12.380738,9.820867,7.850411,5.640958,6.341397,14.308956,10.600959,5.962886,...,6.067222,1.962886,6.953841,9.189779,6.800829,3.100389,3.962886,10.791340,5.498939,7.033275
GSM2445717,14.463665,8.434654,11.657046,9.309123,7.615226,6.112725,7.228203,15.653822,12.572157,5.112725,...,4.228203,6.794549,6.794549,7.434654,6.228203,4.434654,3.697688,10.212512,7.636287,6.142473
GSM2445718,15.110351,8.669961,12.850125,11.105658,9.165918,5.969521,6.554484,13.517701,8.843990,5.706487,...,5.291449,0.384559,6.999269,7.724409,6.191914,2.384559,2.706487,10.028415,6.554484,6.554484
GSM2445719,16.324661,9.475534,11.687919,11.267292,8.913655,6.521337,6.521337,14.222933,10.775579,5.743730,...,3.521337,2.936375,8.332980,8.184302,6.379318,6.276225,4.258303,10.495752,8.498617,5.913655
GSM2445720,17.424133,10.225295,12.963674,11.127778,10.749266,7.215834,7.552869,14.333136,11.267115,6.775262,...,5.552869,3.427338,9.404618,9.841351,7.001330,6.410850,5.360224,9.682152,8.987497,6.253309


In [13]:
# z-score transformation (for PCA)

scaler = StandardScaler()
scaled_data = scaler.fit_transform(imputed_data)

scaled_data = pd.DataFrame(scaled_data).head()
scaled_data.head()


,0,1,2,3,4,5,6,7,8,9,...,707,708,709,710,711,712,713,714,715,716
0,-1.617571,1.047019,-0.316805,0.126234,1.465047,1.076981,1.348911,-0.101085,0.395017,1.089163,...,1.075513,-0.359774,-0.138836,-0.029595,0.508826,0.312080,1.563682,-1.310078,0.733984,1.442897
1,-1.687721,0.873338,-0.505456,0.036140,1.443165,1.153772,1.446995,0.085044,0.654203,0.937254,...,0.524590,-0.067829,-0.152307,-0.128381,0.412915,0.801281,1.419934,-1.365212,1.308240,1.108510
2,-1.642757,0.902163,-0.202076,0.325593,1.570699,1.131767,1.374671,-0.221239,0.099552,1.048506,...,0.843112,-0.455142,-0.134994,-0.112072,0.406836,0.049624,0.882666,-1.382748,1.017585,1.263170
3,-1.564465,0.992506,-0.497091,0.348402,1.552308,1.211399,1.370767,-0.113734,0.420548,1.054645,...,0.312832,-0.300953,-0.022201,-0.086187,0.438226,1.476484,1.723809,-1.338233,1.539927,1.022617
4,-1.499503,1.067009,-0.175144,0.328742,1.671327,1.295501,1.477823,-0.097547,0.489449,1.198003,...,0.921426,-0.271287,0.068429,0.007079,0.542409,1.525844,2.321093,-1.415730,1.671278,1.150115


In [14]:
# export

# imputed log2 transformed data
imputed_data.to_csv('../data/imputed_data.csv')

# normalized version of above^ (for PCA)
scaled_data.to_csv('../data/scaled_data.csv')

# true labels (response y/n)
labels.to_csv('../data/labels.csv')